In [19]:
pip install torch torchvision matplotlib

In [20]:
from torchvision import datasets
from torchvision.transforms import ToTensor
from sklearn.metrics import precision_score, recall_score


In [21]:
train_data = datasets.MNIST(
    root = 'data',
    train = True ,
    transform = ToTensor(),
    download = True
    )


test_data = datasets.MNIST(
    root = 'data',
    train = False ,
    transform = ToTensor(),
    download = True
    )

In [22]:
from torch.utils.data import DataLoader

loaders = {
    'train': DataLoader(train_data, batch_size=100, shuffle=True, num_workers=1),
    'test': DataLoader(test_data, batch_size=100, shuffle=True, num_workers=1),
}

In [23]:
loaders


{'train': <torch.utils.data.dataloader.DataLoader at 0x78024f530a90>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x780250245750>}

In [24]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):

  def __init__(self):
    super(CNN, self).__init__()

    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d() #normalization, deactivates layer
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10) # 10 because 10 digits and softmax function will be used - prob for every digit

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)

    return F.softmax(x, dim=1)


In [25]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(loaders['train']):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()

    if batch_idx % 50 == 0 :
      print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx / len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')


def test():
  model.eval()

  test_loss = 0
  correct = 0

  all_preds = []
  all_targets = []

  with torch.no_grad():
    for data, target in loaders['test']:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += loss_fn(output, target).item()
      pred = output.argmax(dim=1, keepdim = True)
      correct += pred.eq(target.view_as(pred)).sum().item()

      all_preds.extend(pred.cpu().numpy())
      all_targets.extend(target.cpu().numpy())

  precision = precision_score(all_targets, all_preds, average='macro')
  recall = recall_score(all_targets, all_preds, average='macro')

  print(f'Precision: {precision:.4f}, Recall: {recall:.4f}')

  test_loss /= len(loaders['test'].dataset)
  print(f'\nTest set: Average loss: {test_loss: .4f}, Accuracy {correct}/{len(loaders["test"].dataset)} ({100. * correct/len(loaders["test"].dataset):.0f}%\n)')


In [26]:
for epoch in range(1, 11):
  train(epoch)
  test()

<ipython-input-24-e2395ce73220>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	2.304752
Train Epoch: 1 [5000/60000 (8%)]	2.091468
Train Epoch: 1 [10000/60000 (17%)]	1.815430
Train Epoch: 1 [15000/60000 (25%)]	1.779651
Train Epoch: 1 [20000/60000 (33%)]	1.682752
Train Epoch: 1 [25000/60000 (42%)]	1.631898
Train Epoch: 1 [30000/60000 (50%)]	1.627197
Train Epoch: 1 [35000/60000 (58%)]	1.646579
Train Epoch: 1 [40000/60000 (67%)]	1.645466
Train Epoch: 1 [45000/60000 (75%)]	1.623046
Train Epoch: 1 [50000/60000 (83%)]	1.709655
Train Epoch: 1 [55000/60000 (92%)]	1.599739
Precision: 0.9333, Recall: 0.9328

Test set: Average loss:  0.0153, Accuracy 9334/10000 (93%
)
Train Epoch: 2 [0/60000 (0%)]	1.566857


<ipython-input-24-e2395ce73220>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 2 [5000/60000 (8%)]	1.580302
Train Epoch: 2 [10000/60000 (17%)]	1.618145
Train Epoch: 2 [15000/60000 (25%)]	1.609087
Train Epoch: 2 [20000/60000 (33%)]	1.573242
Train Epoch: 2 [25000/60000 (42%)]	1.610754
Train Epoch: 2 [30000/60000 (50%)]	1.572949
Train Epoch: 2 [35000/60000 (58%)]	1.563399
Train Epoch: 2 [40000/60000 (67%)]	1.532866
Train Epoch: 2 [45000/60000 (75%)]	1.565576
Train Epoch: 2 [50000/60000 (83%)]	1.600026
Train Epoch: 2 [55000/60000 (92%)]	1.617979
Precision: 0.9525, Recall: 0.9523

Test set: Average loss:  0.0151, Accuracy 9527/10000 (95%
)
Train Epoch: 3 [0/60000 (0%)]	1.546927


<ipython-input-24-e2395ce73220>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


...

Train Epoch: 8 [5000/60000 (8%)]	1.492774
Train Epoch: 8 [10000/60000 (17%)]	1.512595
Train Epoch: 8 [15000/60000 (25%)]	1.543322
Train Epoch: 8 [20000/60000 (33%)]	1.500460
Train Epoch: 8 [25000/60000 (42%)]	1.545646
Train Epoch: 8 [30000/60000 (50%)]	1.512477
Train Epoch: 8 [35000/60000 (58%)]	1.544722
Train Epoch: 8 [40000/60000 (67%)]	1.501861
Train Epoch: 8 [45000/60000 (75%)]	1.507082
Train Epoch: 8 [50000/60000 (83%)]	1.526539
Train Epoch: 8 [55000/60000 (92%)]	1.502506
Precision: 0.9719, Recall: 0.9721

Test set: Average loss:  0.0149, Accuracy 9720/10000 (97%
)
Train Epoch: 9 [0/60000 (0%)]	1.510419


<ipython-input-24-e2395ce73220>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 9 [5000/60000 (8%)]	1.555480
Train Epoch: 9 [10000/60000 (17%)]	1.571538
Train Epoch: 9 [15000/60000 (25%)]	1.513664
Train Epoch: 9 [20000/60000 (33%)]	1.499050
Train Epoch: 9 [25000/60000 (42%)]	1.512058
Train Epoch: 9 [30000/60000 (50%)]	1.585837
Train Epoch: 9 [35000/60000 (58%)]	1.491888
Train Epoch: 9 [40000/60000 (67%)]	1.560326
Train Epoch: 9 [45000/60000 (75%)]	1.491927
Train Epoch: 9 [50000/60000 (83%)]	1.532446
Train Epoch: 9 [55000/60000 (92%)]	1.521052
Precision: 0.9730, Recall: 0.9730

Test set: Average loss:  0.0149, Accuracy 9730/10000 (97%
)


<ipython-input-24-e2395ce73220>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 10 [0/60000 (0%)]	1.495480
Train Epoch: 10 [5000/60000 (8%)]	1.525319
Train Epoch: 10 [10000/60000 (17%)]	1.502686
Train Epoch: 10 [15000/60000 (25%)]	1.513064
Train Epoch: 10 [20000/60000 (33%)]	1.536533
Train Epoch: 10 [25000/60000 (42%)]	1.516859
Train Epoch: 10 [30000/60000 (50%)]	1.516126
Train Epoch: 10 [35000/60000 (58%)]	1.511727
Train Epoch: 10 [40000/60000 (67%)]	1.505105
Train Epoch: 10 [45000/60000 (75%)]	1.497818
Train Epoch: 10 [50000/60000 (83%)]	1.548641
Train Epoch: 10 [55000/60000 (92%)]	1.523689
Precision: 0.9734, Recall: 0.9733

Test set: Average loss:  0.0149, Accuracy 9734/10000 (97%
)


In [27]:
import os
import torch
import torchvision.transforms as transforms
from PIL import Image

# Path to the folder containing custom handwritten digit images
custom_data_path = "my_digits/"

# Image transformation to match MNIST format (28x28, grayscale, tensor)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure image is in grayscale
    transforms.Resize((28, 28)),                  # Resize to 28x28 pixels
    transforms.ToTensor(),                        # Convert image to tensor
    transforms.Normalize((0.1307,), (0.3081,))     # Normalize as in MNIST
])

# Lists to store images and labels
custom_images = []
custom_labels = []

# Debug: list files in the folder
files_in_dir = os.listdir(custom_data_path)
print("Files found:", files_in_dir)

# Iterate through all PNG files in the folder
for file in files_in_dir:
    if file.lower().endswith(".png"):
        img_path = os.path.join(custom_data_path, file)
        try:
            # Open and transform the image
            img = Image.open(img_path)
            img = transform(img)
            custom_images.append(img)

            # Extract label from the filename (e.g., "0.1.png" -> label 0)
            label = int(file.split('.')[0])
            custom_labels.append(label)
        except Exception as e:
            print(f"Error loading {img_path}: {e}")

# Ensure that images were successfully loaded
if not custom_images:
    raise RuntimeError("No images were loaded. Please verify the folder path and file formats.")

# Convert lists to PyTorch tensors
custom_images = torch.stack(custom_images)
custom_labels = torch.tensor(custom_labels)

print(f"Loaded {len(custom_images)} custom handwritten digit images.")

# Set the model to evaluation mode (assuming `model` and `device` are already defined)
model.eval()

# Move custom images and labels to the same device as the model
custom_images = custom_images.to(device)
custom_labels = custom_labels.to(device)

# Obtain predictions from the model
with torch.no_grad():
    outputs = model(custom_images)
    predictions = outputs.argmax(dim=1)

# Calculate accuracy
correct = (predictions == custom_labels).sum().item()
accuracy = 100. * correct / len(custom_labels)
print(f"Recognition accuracy on custom handwritten digits: {accuracy:.2f}%")

# Analyze and print misclassifications
for i in range(len(custom_labels)):
    true_label = custom_labels[i].item()
    predicted_label = predictions[i].item()
    if true_label != predicted_label:
        print(f"🔴 ERROR: True digit: {true_label}, Model predicted: {predicted_label}")


Files found: ['9.2.png', '7.1.png', '7.2.png', '6.3.png', '.ipynb_checkpoints', '9.3.png', '8.1.png', '4.2.png', '1.1.png', '2.3.png', '3.3.png', '2.1.png', '2.2.png', '5.1.png', '3.1.png', '0.3.png', '6.2.png', '0.2.png', '6.1.png', '5.3.png', '4.3.png', '3.2.png', '1.3.png', '8.3.png', '1.2.png', '8.2.png', '5.2.png', '4.1.png', '0.1.png', '7.3.png', '9.1.png']
Loaded 30 custom handwritten digit images.
Recognition accuracy on custom handwritten digits: 20.00%
🔴 ERROR: True digit: 9, Model predicted: 8
🔴 ERROR: True digit: 7, Model predicted: 8
🔴 ERROR: True digit: 7, Model predicted: 8
🔴 ERROR: True digit: 6, Model predicted: 0
🔴 ERROR: True digit: 9, Model predicted: 8
🔴 ERROR: True digit: 8, Model predicted: 0
🔴 ERROR: True digit: 4, Model predicted: 8
🔴 ERROR: True digit: 1, Model predicted: 8
🔴 ERROR: True digit: 2, Model predicted: 3
🔴 ERROR: True digit: 2, Model predicted: 3
🔴 ERROR: True digit: 5, Model predicted: 8
🔴 ERROR: True digit: 3, Model predicted: 5
🔴 ERROR: True dig

<ipython-input-24-e2395ce73220>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


In [ ]:
import numpy as np
from torchvision import datasets, transforms
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load MNIST dataset using torchvision
train_dataset = datasets.MNIST(
    root='data',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)
test_dataset = datasets.MNIST(
    root='data',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

# Convert dataset images and labels to numpy arrays.
# Flatten each 28x28 image to a vector of size 784.
X_train = train_dataset.data.numpy().reshape(-1, 28*28)
y_train = train_dataset.targets.numpy()
X_test = test_dataset.data.numpy().reshape(-1, 28*28)
y_test = test_dataset.targets.numpy()

# Create and train the Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# Predict on the test set
y_pred = rf.predict(X_test)

# Compute performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')

print(f"Recognition Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity (Recall): {recall:.4f}")


Recognition Accuracy: 0.9705
Precision: 0.9704
Sensitivity (Recall): 0.9702
